# ChatGPT Guide from DeepLearning.AI
link: https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/5/inferring

# Set OpenAI Key
Set the key as env var or in .env file. See file .env.example for an example format.

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    # It turns out there is only one choice ever returned, so this is unnecessary.
    responses = [choice.message["content"] for choice in response.choices]
    return responses

# Sentiment Analysis

In [3]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

In [5]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: ```{lamp_review}```
"""
responses = get_completion(prompt)
for response in responses:
    print(response)

The sentiment of the product review is positive.


In [10]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as one or two words out of the options:
highly positive, semi positive, neutral, semi negative, highly negative

Review text: '''{lamp_review}'''
"""
responses = get_completion(prompt)
for response in responses:
    print(response)

highly positive


In [8]:
# Model changes its answer when asked to consider counterpoints.

prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as one or two words out of the options:
highly positive, semi positive, neutral, semi negative, highly negative

Also indicate any points in the review which contradict your answer.

Review text: ```{lamp_review}```
"""
responses = get_completion(prompt)
for response in responses:
    print(response)

semi positive

Contradiction: The review mentions two issues with the product (broken string and missing part), but overall the sentiment is still positive due to the company's quick and helpful customer service.


# Emotional content

In [12]:
prompt = f"""
Identify a list of emotions that the writer of the 
following review is expressing. Include no more than 
five items in the list. Format your answer as a list of 
lower-case words separated by commas.

Review text: ```{lamp_review}```
"""
responses = get_completion(prompt)
for response in responses:
    print(response)

satisfied, grateful, impressed, content, pleased


In [13]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: ```{lamp_review}```
"""
responses = get_completion(prompt)
for response in responses:
    print(response)

No


In [14]:
prompt = f"""
Is the writer of the following review disappointed in the multiple issues with the product?
The review is delimited with triple backticks. 
Give your answer as either yes or no.

Review text: ```{lamp_review}```
"""
responses = get_completion(prompt)
for response in responses:
    print(response)

No


# Information Extraction

In [15]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: ```{lamp_review}```
"""
responses = get_completion(prompt)
for response in responses:
    print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


# Multiple Tasks

In [16]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: ```{lamp_review}```
"""
responses = get_completion(prompt)
for response in responses:
    print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp with additional storage",
  "Brand": "Lumina"
}


# K Classification: topic

In [17]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

In [23]:
prompt = f"""
Determine the five main topics that are being discussed in the 
following text, which is delimited by triple backticks. Don't mention minor topics.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: ```{story}```
"""
responses = get_completion(prompt)
for response in responses:
    print(response.split(sep=','))

['government survey', ' public sector employees', ' job satisfaction', ' NASA', ' Social Security Administration']


# Topic detection

In [24]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [25]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: ```{story}```
"""

responses = get_completion(prompt)
for response in responses:
    print(response)

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


In [29]:
for response in responses:
    topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
    for topic in topic_dict:
        if topic_dict[topic] == 1:
            print(f"ALERT: New {topic} story!")

ALERT: New nasa story!
ALERT: New employee satisfaction story!
ALERT: New federal government story!
